<a href="https://colab.research.google.com/github/galbendavids/galbendavids/blob/main/dicta_heb_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this notebook is being used for me
as i try many hebrew LLM models for my role at tocndix

In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.37.2
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
  Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:0

In [ ]:
import transformers
transformers.__version__

In [ ]:
#
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "dicta-il/dictalm-7b-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
file_path = 'https://www.dropbox.com/scl/fi/450wrvtp3yxk6oe2bwnfs/3_articles.csv?rlkey=ze97161x6auiwf8hh0rvcdqnc&dl=1'
df = pd.read_csv(file_path)
df.head()

,text
0,"המסע של לילי גלדסטון, ""הנערה מהשמורה"", עוד יסת..."
1,עוגיות קשיו עדינות למה לא משלבים מספיק קשיו בא...
2,"תבשיל עוף הציידים של טוסקנה תבשיל עוף, יין אדו..."


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('dicta-il/dictalm-7b-instruct')
# model = AutoModelForCausalLM.from_pretrained('dicta-il/dictalm-7b-instruct', trust_remote_code=True, load_in_8bit=True).cuda()

model.eval()

with torch.inference_mode():
  prompt = df.loc[1, 'text']
  prompt +="מה המרכיבים של המנה?"
  kwargs = dict(
        inputs=tokenizer(prompt, return_tensors='pt').input_ids.to(model.device),
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9,
        max_length=2000,
        min_new_tokens=5
    )

  print(tokenizer.batch_decode(model.generate(**kwargs), skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

['עוגיות קשיו עדינות למה לא משלבים מספיק קשיו באפייה? סער מור יצר עוגיות נהדרות, רכות ומתפוררות עם גוון טעם עדין של קשיו הידעתם שאגוז הקשיו הוא קרוב משפחה של הפיסטוק, אך אינו נחשב מבחינה בוטנית לאגוז אלא כפרי? אני מאוד מחבב אגוזי קשיו, ליתר דיוק את פרי הקשיו המיובש. לטעמי לא עושים בו מספיק שימוש באפייה. לכן בחרתי בו להכנת עוגיות נהדרות, רכות ופירוריות מעט עם גוון טעם עדין של קשיו.כמובן שאני חייב לשחק במרקמים וטעמים ולצורך כך ועל מנת להישאר נאמן לקשיו, קניתי אותו בחנות הפיצוחים בשלושה מצבי צבירה: טבעי, קלוי ומעט מומלח ומקורמל, מה שנקרא "מסוכר". את שלושת הסוגים הללו הכנסתי לבצק מאחר שכל אחד מהם מעניק גוון טעם ומרקם מעט שונה.לגבי אפיית העוגיות. הן תופחות ומכפילות את נפחן בתנור, השאירו מרווחים בין העוגיות בתבנית. אני אופה אותן במשך 13 דקות כך שהן מקבלות מעטפת פריכה דקה מבחוץ ומעט לחות מבפנים. ככה אני אוהב אותן. יש האופים אותן עוד 2-3 דקות והם הופכות לקשות ופריכות יותר, לשיקולכם. בכל מקרה, חשוב שלא לאפות אפיית יתר. אם ישחימו יתר על המידה הן פחות טעימות.חומרים לכ-20 עוגיות:120 גרם תערובת אגו